In [17]:
from langchain_community.llms import Ollama

llm = Ollama(model = 'EEVE-Korean-10.8B:latest')

In [18]:
llm.invoke("대답은 짧게 해줘")

'물론이죠, 도움이 되고 존중하는 조수로서 여러분의 질문에 최선을 다해 답변해드리겠습니다! 정확하고 관련 있는 답변을 제공하기 위해 노력하면서 동시에 안전하고 사회적으로 편견이 없는 반응을 보장하겠습니다. 궁금한 점이나 우려사항이 있으시면 언제든지 문의해주세요.\r\n\r\n짧은 대답을 원하신다면 다음과 같습니다:\r\n\r\n1. "인공지능 조수로서 저는 여러분을 도와드리기 위해 여기에 있습니다! 질문을 하거나 저에게 과제를 주십시오."\r\n2. "저는 AI 어시스턴트로서 정확하고 유용한 정보를 제공하기 위해 노력합니다. 궁금한 점이나 우려사항이 있으면 언제든지 물어보세요!"\r\n3. "긍정적이고 참여적인 태도를 유지하며 여러분을 돕기 위해 최선을 다하겠습니다. 무엇이든 도와드릴 준비가 되어 있습니다."\r\n4. "당신의 질문을 듣고 있으며, 답변을 드리기 위해 노력하고 있습니다. 조금만 기다려 주세요."\r\n5. "AI 조수로서 제가 할 수 있는 모든 방법으로 여러분에게 도움이 되고자 합니다. 궁금한 점이나 우려사항이 있으면 알려주세요!"\r\n6. "여러분을 지원하고 최선을 다해 여러분의 필요를 충족시키기 위해 여기에 있습니다. 무엇이든 도와드릴 준비가 되어 있습니다."\r\n7. "저에게는 항상 질문이 있거나 해결해야 할 문제가 있다면 도움을 줄 준비가 되어 있는 AI 어시스턴트가 있습니다. 망설이지 말고 물어보세요!"\r\n8. "저는 여러분을 돕기 위해 여기에 있으며, 여러분의 모든 질문과 우려사항을 해결하는 데 최선을 다할 것입니다. 무엇이든 도와드릴 준비가 되어 있습니다."\r\n9. "여러분이 저를 의지하는 것이 얼마나 중요한지 알고 있으며, 항상 최선을 다해 여러분을 지원하고자 합니다. 궁금한 점이나 우려사항이 있으면 언제든지 물어보세요!"\r\n10. "저는 여러분의 AI 어시스턴트로서 정확하고 유용한 정보를 제공하기 위해 노력합니다. 무엇이든 도와드릴 준비가 되어 있으니 망설이지 말고 물어보세요!"'

In [19]:
# Load Data
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_path="https://blog.langchain.dev/langgraph/"
)

docs = loader.load()

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True
)

In [21]:
all_splits = text_splitter.split_documents(docs)

In [22]:
# index the data
from langchain_community import embeddings

embedding = embeddings.ollama.OllamaEmbeddings(
    # model="nomic-embed-text:latest"
    model = "EEVE-Korean-10.8B:latest"
)

In [23]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding
)

In [25]:
retriver = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 6}
)

In [26]:
retriver.get_relevant_documents("What is LangGraph?")

[Document(page_content='some state)More controlled human-in-the-loop workflowsMulti-agent workflowsIf any of these resonate with you, please feel free to add an example notebook in the LangGraph repo, or reach out to us at hello@langchain.dev for more involved collaboration!', metadata={'language': 'en', 'source': 'https://blog.langchain.dev/langgraph/', 'start_index': 11185, 'title': 'LangGraph'}),
 Document(page_content='Tags\nBy LangChain\n\n\nJoin our newsletter\nUpdates from the LangChain team and community\n\n\nEnter your email\n\nSubscribe\n\nProcessing your application...\nSuccess! Please check your inbox and click the link to confirm your subscription.\nSorry, something went wrong. Please try again.\n\n\n\n\n\nYou might also like\n\n\n\n\n\n\n\n\n\n\nReflection Agents\n\n\nagents\n6 min read\n\n\n\n\n\n\n\n\n\n\n\n\nPlan-and-Execute Agents\n\n\nBy LangChain\n5 min read\n\n\n\n\n\n\n\n\n\n\n\n\n[Week of 2/19] LangChain Release Notes\n\n\nBy LangChain\n1 min read\n\n\n\n\n\n\n\n

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """You must answer in Korean."""

In [28]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [29]:
from langchain_core.output_parsers import StrOutputParser

contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [30]:
from langchain_core.messages import AIMessage, HumanMessage

contextualize_q_chain.invoke(
    {
        "chat_history":[
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large Language model")
        ],
        "question" : "What is meant by large?"
    }
)

'대형 모델이란 언어 이해와 생성 능력을 향상시키기 위해 대량의 데이터셋을 학습한 AI 시스템을 의미합니다.\r\n\r\n예를 들어, LLM은 문맥과 구조를 고려하여 질문에 답하고, 텍스트를 생성하며, 복잡한 작업을 수행하기 위해 자연어 처리(Natural Language Processing, NLP)와 기계학습(Machine Learning, ML) 기술을 사용하는 첨단 AI 시스템입니다. 이 모델들은 대량의 데이터를 학습함으로써 언어 능력을 향상시키고 다양한 분야에서 인간과 같은 수준의 성능을 달성할 수 있습니다.\r\n\r\n대형 모델은 일반적으로 대규모 사전 훈련된 모델로, 방대한 데이터셋에서 미리 학습되어 새로운 작업이나 도메인에 쉽게 적용될 수 있습니다. 이러한 모델들은 복잡한 작업을 수행하고 인간의 언어 능력을 시뮬레이션하는 데 탁월한 성과를 보여줍니다.\r\n\r\nLLM 중 대표적인 예시로는 BERT, GPT-3, T5 등이 있으며, 이들 모두 자연어 처리 분야에서 중요한 발전을 이루었습니다.'

In [31]:
qa_system_prompt = """You must answer in Korean.\
대답한 최대한 짧게 만들어줘\

{context}"""

In [32]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [33]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [34]:
def contextualize_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]

In [35]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    RunnablePassthrough.assign(
        context = contextualize_question | retriver | format_docs
    )
    | qa_prompt
    | llm
)

In [36]:
chat_history = []

question = "what is LangGraph?"
ai_msg = rag_chain.invoke(
    {
        "question": question,
        "chat_history": chat_history
    }
)

In [37]:
ai_msg

'LangGraph는 인간-루프 워크플로우(human-in-the-loop workflows), 다중 에이전트 워크플로우(multi-agent workflows)를 포함한 다양한 고급 기능을 지원하는 랭체인(LangChain)의 확장판입니다.\r\n\r\nLangGraph는 LangChain 생태계 내에서 작업 흐름을 관리하고 조정하는 데 사용되며, 언어 모델을 사용하여 여러 에이전트를 포함하고 복잡한 작업을 처리하는 능력을 제공합니다. 이는 워크플로우를 설정하고 관리하는 데 있어 더 세밀한 제어와 유연성을 가능하게 합니다.\r\n\r\n예를 들어, LangGraph를 통해 사용자는 기존 LangChain 에이전트들과 함께 새로운 노드를 생성하여 맞춤형 워크플로를 만들 수 있습니다. 또한 이러한 워크플로에서 다양한 언어 모델과 서비스를 활용할 수 있어 더욱 정교하고 강력한 솔루션을 개발할 수 있게 됩니다.\r\n\r\nLangGraph는 랭체인 팀 및 커뮤니티와의 협업 기회를 제공하며, 사용자는 LangGraph 저장소에 예제 노트북을 추가하거나 hello@langchain.dev로 더 깊은 협업을 제안할 수도 있습니다.\r\n\r\n요약하자면, LangGraph는 고급 기능 세트를 갖춘 강력한 도구로서, 사용자가 여러 에이전트와 복잡한 워크플로우를 포함하는 맞춤형 언어 모델 기반 시스템을 구축할 수 있게 합니다.'

In [39]:
chat_history.extend(
    [
        HumanMessage(content=question), ai_msg
    ]
)

In [40]:
second_question = "What is it used for?"

rag_chain.invoke(
    {
        "question": second_question,
        "chat_history": chat_history
    }
)

'LangGraph는 랭체인(LangChain)의 확장판으로, 인간-루프 워크플로우(human-in-the-loop workflows), 다중 에이전트 워크플로우(multi-agent workflows)를 포함한 다양한 고급 기능을 지원합니다.\r\n\r\nLangGraph는 LangChain 생태계 내에서 작업 흐름을 관리하고 조정하는 데 사용되며, 언어 모델을 사용하여 여러 에이전트를 포함하고 복잡한 작업을 처리할 수 있게 합니다. 이는 워크플로우 설정을 위한 더 세밀한 제어와 유연성을 제공합니다.\r\n\r\n예를 들어, LangGraph를 통해 사용자는 기존 LangChain 에이전트들과 함께 새로운 노드를 생성하여 맞춤형 워크플로를 만들 수 있습니다. 또한 이러한 워크플로에서 다양한 언어 모델과 서비스를 활용할 수 있어 더욱 정교하고 강력한 솔루션을 개발할 수 있게 됩니다.\r\n\r\nLangGraph는 랭체인 팀 및 커뮤니티와의 협업 기회를 제공하며, 사용자는 LangGraph 저장소에 예제 노트북을 추가하거나 hello@langchain.dev로 더 깊은 협업을 제안할 수도 있습니다.\r\n\r\n요약하자면, LangGraph는 고급 기능 세트를 갖춘 강력한 도구로서, 사용자가 여러 에이전트와 복잡한 워크플로우를 포함하는 맞춤형 언어 모델 기반 시스템을 구축할 수 있게 합니다.'